<a href="https://colab.research.google.com/github/J1116/physics/blob/main/atom_light_interaction_ver3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import math
import plotly
import plotly.graph_objects as go
import plotly.io as pio
pio.renderers.default = "colab"
print(plotly.__version__)
#plotly.offline.init_notebook_mode(connected=False)
from sympy.physics.wigner import wigner_6j
from scipy import optimize
from scipy.optimize import curve_fit

4.4.1


In [ ]:
# fundamental constants
pi = math.pi
kB = 1.380e-23 # Boltzmann constant in J/K 
Na = 6.022e23 # Avogadro number 
c = 2.9979e8 # speed of light in m/s 
muB = 9.2741e-24 # Bohr magneton in J/T 
muN = 5.051e-27 # nuclear magneton in J/T 
hP = 6.6262e-34 # Planck constant in J.s
hbar = hP/(2*pi) # in J.s
re = 2.816e-15 # classical electron radius in m 
amg = 2.652e25 # Loschmidt's constant (amagat) in m^{-3}
eps0 = 8.8541e-12 #permittivity of vaccum in F/m
ec = 1.6e-19

In [ ]:
# probe light parameters
Dprobe = 3.1e-3 # probe beam e^-2 diameter in m

Pprobe = 15e-3 # probe light power in W
Iprobe = 8 / (pi * Dprobe ** 2) * Pprobe # probe intensity at the center in W/m^2

In [ ]:
# detuning
Delta_exp = 2*pi * -1e8 # in rad/s
Delta1 = Delta_exp + 2*pi*509e6
Delta2 = Delta_exp - 2*pi*305e6

In [ ]:
# polarization angle
thetaD = 54.7357 # in degrees
theta = pi/2 - math.asin(1/math.sqrt(3)) # in rad

In [ ]:
# pulse configuration
thold = 0e-6 # hold time in s
nframe = 5 # number of frames
nframeB = 1
npulse = 35
period = 30e-6 # period of probe pulses in s
period2 = period / nframeB
width = 0.5e-6 # pulse width in s
tau = width / nframe

In [ ]:
# Rb-87 properties
S = 1/2 # electron spin
I = 3/2 # nuclear spin
# J: electron's total spin in an excited state
J = 1/2 # for D1 line
F = 2; # total spin
F21 = 1
F22 = 2

In [ ]:
# calculate multiplicities
gS = 2*S + 1;
gI = 2*I + 1;
gJ = 2*J + 1;
gF = 2*F + 1;
gg = gI * gS # multiplicity of ground state
ge = gI * gJ # multiplicity of excited state

In [ ]:
MW = 86.9 # molcular weight for Rb in grams/mol
LgS = 2.00231 # Lande g-value of S1/2 state
LgJ = gJ / 3 # approx. Lande g-value of PJ state
muI = 2.751 * muN # nuclear moment in J/T
Ag = hP * 3417.35e6 # S1/2 dipole coupling coef. in J

In [ ]:
if J == 1.5:
    lamJ = 780e-9 # D2 wavelength in m
    Ae = hP * 84.852e6 # P3/2 dipole coefficient in erg
    Be = hP * 12.611e6 # P3/2 quadrupole coupling coefficient in J
    te = 25.5e-9 # spontaneous P1/2 lifetime in s
elif J == 0.5:
    lamJ = 795e-9 # D1 wavelength in m
    Ae = hP * 409e6 # P1/2 dipole coupling coefficient in J
    te = 28.5e-9 # spontaneous P1/2 lifetime in s
    decay = 3.6129e7 #spontaneous emission rate

In [ ]:
keg = 2*pi / lamJ;
weg = c*keg # nominal spatial and temporal frequencies
feg = c*gJ / ( 4*weg**2 * re * te ) # oscillator strength
D = math.sqrt( gS * hbar * re * c**2 * feg / (2*weg) ) # dipole moment in esu cm
radius = np.zeros(3) #radius of atomic ensemble in m
radius[0] = 4e-6
radius[1] = 4e-6
radius[2] = 50e-6
vol = 2 * pi * radius[0] * radius[1] * radius[2] /3 #volume of atomic ensemble in m^3
NA = 3e5 #atom number
B0 = 140e-7 #magnetic field in T
power = Iprobe * pi * radius[0] ** 2 
NL = power * width / (hbar * weg) ##photon number
NL2 = NL / nframe
print(NL)

399791.25645946705


In [ ]:
## spin matrices and operators
# spin matrices in uncoupled basis
# spin orientation
sFp = np.diag(np.ones(2*F), k=1)
sFm = sFp.T
sFo = np.zeros((gF,gF,3), dtype=np.complex128)
sFo[:,:,0] = (sFp+sFm)/math.sqrt(2) # x
sFo[:,:,1] = -1j*(sFp-sFm)/math.sqrt(2) # y
sFo[:,:,2] = np.diag(np.arange(F,-F-1,-1)) # z

# spin alignment
sJa = np.zeros((gF,gF,5), dtype=np.complex128)
sJa[:,:,0] = np.linalg.matrix_power(sFo[:,:,0], 2) - np.linalg.matrix_power(sFo[:,:,1], 2)
sJa[:,:,1] = np.dot(sFo[:,:,0], sFo[:,:,1]) + np.dot(sFo[:,:,1], sFo[:,:,0]) 
sJa[:,:,2] = np.dot(sFo[:,:,0], sFo[:,:,2]) + np.dot(sFo[:,:,2], sFo[:,:,0])
sJa[:,:,3] = np.dot(sFo[:,:,1], sFo[:,:,2]) + np.dot(sFo[:,:,2], sFo[:,:,1])
sJa[:,:,4] = (2*np.linalg.matrix_power(sFo[:,:,2], 2) - np.linalg.matrix_power(sFo[:,:,0], 2) - np.linalg.matrix_power(sFo[:,:,1], 2))/math.sqrt(3)


In [ ]:
sJa[:,:,3]

array([[0.+0.j        , 0.-2.12132034j, 0.+0.j        , 0.+0.j        ,
        0.+0.j        ],
       [0.+2.12132034j, 0.+0.j        , 0.-0.70710678j, 0.+0.j        ,
        0.+0.j        ],
       [0.+0.j        , 0.+0.70710678j, 0.+0.j        , 0.+0.70710678j,
        0.+0.j        ],
       [0.+0.j        , 0.+0.j        , 0.-0.70710678j, 0.+0.j        ,
        0.+2.12132034j],
       [0.+0.j        , 0.+0.j        , 0.+0.j        , 0.-2.12132034j,
        0.+0.j        ]])

In [ ]:
print(np.dot(sFo[:,:,1], sFo[:,:,2]) - np.dot(sFo[:,:,2], sFo[:,:,1]))
print(sFo[:,:,0])

[[0.+0.j         0.+0.70710678j 0.+0.j         0.+0.j
  0.+0.j        ]
 [0.+0.70710678j 0.+0.j         0.+0.70710678j 0.+0.j
  0.+0.j        ]
 [0.+0.j         0.+0.70710678j 0.+0.j         0.+0.70710678j
  0.+0.j        ]
 [0.+0.j         0.+0.j         0.+0.70710678j 0.+0.j
  0.+0.70710678j]
 [0.+0.j         0.+0.j         0.+0.j         0.+0.70710678j
  0.+0.j        ]]
[[0.        +0.j 0.70710678+0.j 0.        +0.j 0.        +0.j
  0.        +0.j]
 [0.70710678+0.j 0.        +0.j 0.70710678+0.j 0.        +0.j
  0.        +0.j]
 [0.        +0.j 0.70710678+0.j 0.        +0.j 0.70710678+0.j
  0.        +0.j]
 [0.        +0.j 0.        +0.j 0.70710678+0.j 0.        +0.j
  0.70710678+0.j]
 [0.        +0.j 0.        +0.j 0.        +0.j 0.70710678+0.j
  0.        +0.j]]


In [ ]:
##coupling constants G1 G2
g = weg / (2*eps0)
alp0 = 3*decay*eps0*hbar*lamJ*lamJ*lamJ/(8*pi*pi)
alp11 = alp0*abs(wigner_6j(1,J,J,I,F21,F, prec=32))*abs(wigner_6j(1,J,J,I,F21,F, prec=32))
alp12 = alp0*abs(wigner_6j(1,J,J,I,F22,F, prec=32))*abs(wigner_6j(1,J,J,I,F22,F, prec=32))
alp21 = -1*alp12*(2*F+1)/(F*(F+1))/Delta2 - alp11*(2*F-1)/F/Delta1
alp22 = -1*alp12*(2*F+1)/(F*(F+1))/Delta2 + alp11/F/Delta1
G1 = g*alp21/hbar
G2 = g*alp22/hbar
#G1 = g*alp21/Delta_exp
#G2 = g*alp22/Delta_exp
#G1 = 1.8e-6
#G2 = -9.3e-8
print(G1)
print(G2)

-0.000000026366612468672146520350517884529
0.000000026628315818237129927282683780854


In [ ]:
o21 = np.sqrt(10)*wigner_6j(F21,I,J,J,1,F, prec=32)
c21 = np.sqrt(30)*(2*F21+1)/np.sqrt(F*(F+1)*(2*F+1)*(2*F-1)*(2*F+3)) \
*wigner_6j(F,1,F21,1,F,2, prec=32) * abs(o21)**2
o22 = np.sqrt(10)*wigner_6j(F22,I,J,J,1,F, prec=32)
c22 = np.sqrt(30)*(2*F22+1)/np.sqrt(F*(F+1)*(2*F+1)*(2*F-1)*(2*F+3)) \
*wigner_6j(F,1,F22,1,F,2, prec=32) * abs(o22)**2

In [ ]:
print(c21)
print(c22)

0.083333333333333350253811304774474
0.083333333333333353080300776234252


In [ ]:
abs(wigner_6j(1,J,J,I,F21,F))*abs(wigner_6j(1,J,J,I,F21,F))
abs(wigner_6j(1,J,J,I,F22,F))*abs(wigner_6j(1,J,J,I,F22,F))
(2*F+1)/(F*(F+1))

0.8333333333333334

In [ ]:
print(-1*alp12*(2*F+1)/(F*(F+1))/alp0)
print(alp11/F/alp0)

-0.041666666666666666666666666666667
0.041666666666666666666666666666667


In [ ]:
##structure constants
stcon = np.zeros((12,12,12), dtype=np.float64)
for i in range(12):
    for j in range(12):
        for k in range(12):
            if (i==0 and j==1 and k==2) or (i==1 and j==2 and k==0) or (i==2 and j==0 and k==1):
                stcon[i,j,k] = 1
            elif (i==2 and j==1 and k==0) or (i==0 and j==2 and k==1) or (i==1 and j==0 and k==2):
                stcon[i,j,k] = -1
            elif (i==3 and j==4 and k==2) or (i==2 and j==3 and k==4) or (i==4 and j==2 and k==3):
                stcon[i,j,k] = 2
            elif (i==2 and j==4 and k==3) or (i==4 and j==3 and k==2) or (i==3 and j==2 and k==4):
                stcon[i,j,k] = -2
            elif (i==0 and j==6 and k==7) or (i==7 and j==0 and k==6) or (i==6 and j==7 and k==0) \
                or (i==1 and j==7 and k==5) or (i==5 and j==1 and k==7) or (i==7 and j==5 and k==1):
                stcon[i,j,k] = math.sqrt(3)
            elif (i==7 and j==6 and k==0) or (i==6 and j==0 and k==7) or (i==0 and j==7 and k==6) \
                or (i==5 and j==7 and k==1) or (i==7 and j==1 and k==5) or (i==1 and j==5 and k==7):
                stcon[i,j,k] = -1*math.sqrt(3)
            elif (i==0 and j==4 and k==5) or (i==5 and j==0 and k==4) or (i==4 and j==5 and k==0) \
                or (i==0 and j==6 and k==3) or (i==3 and j==0 and k==6) or (i==6 and j==3 and k==0) \
                or (i==1 and j==5 and k==3) or (i==3 and j==1 and k==5) or (i==5 and j==3 and k==1) \
                or (i==2 and j==5 and k==6) or (i==6 and j==2 and k==5) or (i==5 and j==6 and k==2) \
                or (i==8 and j==9 and k==10) or (i==10 and j==8 and k==9) or (i==9 and j==10 and k==8):
                stcon[i,j,k] = 1
            elif (i==5 and j==4 and k==0) or (i==4 and j==0 and k==5) or (i==0 and j==5 and k==4) \
                or (i==3 and j==6 and k==0) or (i==6 and j==0 and k==3) or (i==0 and j==3 and k==6) \
                or (i==3 and j==5 and k==1) or (i==5 and j==1 and k==3) or (i==1 and j==3 and k==5) \
                or (i==6 and j==5 and k==2) or (i==5 and j==2 and k==6) or (i==2 and j==6 and k==5) \
                or (i==10 and j==9 and k==8) or (i==9 and j==8 and k==10) or (i==8 and j==10 and k==9):
                stcon[i,j,k] = -1

In [ ]:
pra1 = np.array([1,2,3], dtype=np.float64)
print(pra1.dtype)
pra2 = np.array([2+1j,3+2j,4+3j], dtype=np.complex128)
print(pra2.dtype)
pra3 = np.dot(pra1, pra2)
print(pra3)
print(pra3.dtype)

float64
complex128
(20+14j)
complex128


In [ ]:
##light-atom interaction Hamiltonian
Hla = np.zeros((12,12,12), dtype=np.float64)
for i in range(12):
    for j in range(12):
        for k in range(12):
            if j == 10:
                Hla[i,j,k] = G1*stcon[i,2,k]
            elif k == 2:
                Hla[i,j,k] = G1*stcon[i,10,j]
            elif j == 8:
                Hla[i,j,k] = G2*stcon[i,3,k]
            elif j == 9:
                Hla[i,j,k] = G2*stcon[i,4,k]
            elif j == 11:
                Hla[i,j,k] = G2*stcon[i,7,k]
            elif k == 3:
                Hla[i,j,k] = G2*stcon[i,8,j]
            elif k == 4:
                Hla[i,j,k] = G2*stcon[i,9,j]

In [ ]:
##block diagonal matrix for magnetic field
bx = math.cos(theta)
by = math.sin(theta)
A = np.zeros((8,8), dtype=np.float64)
A[0,2] = by
A[1,2] = -bx
A[3,5] = by
A[3,6] = bx
A[4,5] = -bx
A[4,6] = by
A[5,7] = math.sqrt(3)*by
A[6,7] = -1*math.sqrt(3)*bx

for i in range(8):
    for j in range(i, 8):
        if not A[i,j]==0:
            A[j,i] = -A[i,j]
A

array([[ 0.        ,  0.        ,  0.81649658,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        , -0.57735027,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ],
       [-0.81649658,  0.57735027,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.81649658,  0.57735027,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        -0.57735027,  0.81649658,  0.        ],
       [ 0.        ,  0.        ,  0.        , -0.81649658,  0.57735027,
         0.        ,  0.        ,  1.41421356],
       [ 0.        ,  0.        ,  0.        , -0.57735027, -0.81649658,
         0.        ,  0.        , -1.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        -1.41421356,  1.        ,  0.        ]])

In [ ]:
a_pre, P = np.linalg.eig(A)
a = (a_pre - a_pre.conjugate())/2
for i in range(8):
    for j in range(8):
        if abs(P[i,j].real)<=1e-10:
            P[i,j] = (P[i,j] - P[i,j].conjugate())/2
        elif abs(P[i,j].imag)<=1e-10:
            P[i,j] = (P[i,j] + P[i,j].conjugate())/2

P_inv = np.linalg.inv(P)
for i in range(8):
    for j in range(8):
        if abs(P_inv[i,j].real)<=1e-10:
            P_inv[i,j] = (P_inv[i,j] - P_inv[i,j].conjugate())/2
        elif abs(P_inv[i,j].imag)<=1e-10:
            P_inv[i,j] = (P_inv[i,j] + P_inv[i,j].conjugate())/2

#P = np.zeros((8,8,8), dtype=np.complex128)
#for i in range(8):
#    for j in range(8):
#        for k in range(8):
#            P[i,j,k] = v[i,j].conjugate() * v[i,k]
#print(P)
#print(P_inv)
print(a)
#print(np.dot(np.dot(P, np.diag(a)), P_inv))

[0.+1.j 0.-1.j 0.+0.j 0.+2.j 0.-2.j 0.+0.j 0.+1.j 0.-1.j]


In [ ]:
##initial state
V = np.zeros((npulse+1,12), dtype=np.float64)
V[0,:] = np.array([NA*sFo[0,0,0],NA*sFo[0,0,1],NA*sFo[0,0,2],NA*sJa[0,0,0],NA*sJa[0,0,1],NA*sJa[0,0,2],NA*sJa[0,0,3],NA*sJa[0,0,4],NL2/2,0,0,NL2/2])
GammaV = np.zeros((npulse+1,12,12), dtype=np.float64)
Vdel = np.zeros((8,5,5), dtype=np.complex128)
for i in range(8):
    if i<=2:
        Vdel[i,:,:] = sFo[:,:,i] - V[0,i]*np.eye(5)/NA
    elif i>=3:
        Vdel[i,:,:] = sJa[:,:,i-3] - V[0,i]*np.eye(5)/NA
for i in range(12):
    for j in range(12):
        if i<=7 and j<=7:
            GammaV[0,i,j] = NA*(np.dot(Vdel[i,:,:], Vdel[j,:,:])[0,0] + np.dot(Vdel[j,:,:], Vdel[i,:,:])[0,0]) / 2
        elif i>=8 and i<=10 and j>=8 and j<=10:
            GammaV[0,i,i] = NL2/4
L = np.zeros((npulse+1,8),dtype=np.float64)
L[0,:] = V[0,:8]
GammaL = np.zeros((npulse+1,8,8), dtype=np.float64)
GammaL[0,:,:] = GammaV[0,:8,:8]
print(V[0,:])
print(GammaV[0,:,:])
VV = np.zeros((npulse+1,12), dtype=np.float64) 
VV[:,:] = V[:,:]
GammaVV = np.zeros((npulse+1,12,12), dtype=np.float64)
GammaVV[:,:,:] = GammaV[:,:,:]

[      0.               0.          600000.               0.
       0.               0.               0.         1212435.56529821
   39979.12564595       0.               0.           39979.12564595]
[[ 150000.               0.               0.               0.
        0.          450000.               0.               0.
        0.               0.               0.               0.        ]
 [      0.          150000.               0.               0.
        0.               0.          450000.               0.
        0.               0.               0.               0.        ]
 [      0.               0.               0.               0.
        0.               0.               0.               0.
        0.               0.               0.               0.        ]
 [      0.               0.               0.          300000.
        0.               0.               0.               0.
        0.               0.               0.               0.        ]
 [      0.          

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: ComplexWarning:

Casting complex values to real discards the imaginary part

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: ComplexWarning:

Casting complex values to real discards the imaginary part



In [ ]:
print(-B0*muB*LgS*a[0].imag*period/hbar/200)

-0.3697751430421156


In [ ]:
time = np.zeros(npulse+1)
time[0] = 0
X = np.zeros((8,8),dtype=np.complex128)
for j in range(8):
    X[j,j] = math.cos(-B0*muB*LgS*a[j].imag*period2/hbar)+1j*math.sin(-B0*muB*LgS*a[j].imag*period2/hbar)
U = np.dot(np.dot(P, X),P_inv)
U = U.astype(np.float64)
U_T = U.T
#print(U)
#print(U_T)
Y = np.zeros((8,8),dtype=np.complex128)
for j in range(8):
    Y[j,j] = math.cos(-B0*muB*LgS*a[j].imag*tau/hbar)+1j*math.sin(-B0*muB*LgS*a[j].imag*tau/hbar)
U2 = np.dot(np.dot(P, Y),P_inv)
U2 = U2.astype(np.float64)
U2_T = U2.T

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: ComplexWarning:

Casting complex values to real discards the imaginary part

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: ComplexWarning:

Casting complex values to real discards the imaginary part



In [ ]:
#        L[i+1+j,:] = np.dot(U, L[i+j,:])
#        GammaL[i+1+j,:,:] = np.dot(np.dot(U, GammaL[i+j,:,:]), U_T)
#        time[i+1+j] = time[i+j] + period2
#       V[i+1+j,:8] = L[i+1+j,:]
#        V[i+1+j,8:12] = V[0,8:12]
#        GammaV[i+1+j,:8,:8] = GammaL[i+1+j,:,:]
#        GammaV[i+1+j,8:12,8:12] = GammaV[0,8:12,8:12]

In [ ]:
V = np.zeros((101,npulse+1,12), dtype=np.float64)
GammaV = np.zeros((101,npulse+1,12,12), dtype=np.float64)
VV = np.zeros((101,npulse+1,12), dtype=np.float64)
GammaVV = np.zeros((101,npulse+1,12,12), dtype=np.float64)
time = np.zeros((101,npulse+1),dtype=np.float64)
pulse_period = np.zeros(101, dtype = np.float64)

for detune in range(101):
    # detuning
#    Delta_exp = 2*pi * (-1)*(detune+50)*1e7 # in rad/s
#    Delta1 = Delta_exp + 2*pi*509e6
#    Delta2 = Delta_exp - 2*pi*305e6
    period = (20 + (detune*0.5))*1e-6 # period of probe pulses in s
    period2 = period / nframeB
    pulse_period[detune] = period2
    ##coupling constants G1 G2
    g = weg / (2*eps0*vol)
    alp0 = 3*decay*eps0*hbar*lamJ*lamJ*lamJ/(8*pi*pi)
    alp11 = alp0*abs(wigner_6j(1,J,J,I,F21,F, prec=32))*abs(wigner_6j(1,J,J,I,F21,F, prec=32))
    alp12 = alp0*abs(wigner_6j(1,J,J,I,F22,F, prec=32))*abs(wigner_6j(1,J,J,I,F22,F, prec=32))
    alp21 = -1*alp12*(2*F+1)/(F*(F+1))/Delta2 - alp11*(2*F-1)/F/Delta1
    alp22 = -1*alp12*(2*F+1)/(F*(F+1))/Delta2 + alp11/F/Delta1
    G1 = g*alp21*1e20
    G2 = g*alp22*1e20

    ##light-atom interaction Hamiltonian
    Hla = np.zeros((12,12,12), dtype=np.float64)
    for i in range(12):
        for j in range(12):
            for k in range(12):
                if j == 10:
                    Hla[i,j,k] = G1*stcon[i,2,k]
                elif k == 2:
                    Hla[i,j,k] = G1*stcon[i,10,j]
                elif j == 8:
                    Hla[i,j,k] = G2*stcon[i,3,k]
                elif j == 9:
                    Hla[i,j,k] = G2*stcon[i,4,k]
                elif j == 11:
                    Hla[i,j,k] = G2*stcon[i,7,k]
                elif k == 3:
                    Hla[i,j,k] = G2*stcon[i,8,j]
                elif k == 4:
                    Hla[i,j,k] = G2*stcon[i,9,j]

    a_pre, P = np.linalg.eig(A)
    a = (a_pre - a_pre.conjugate())/2
    for i in range(8):
        for j in range(8):
            if abs(P[i,j].real)<=1e-10:
                P[i,j] = (P[i,j] - P[i,j].conjugate())/2
            elif abs(P[i,j].imag)<=1e-10:
                P[i,j] = (P[i,j] + P[i,j].conjugate())/2

    P_inv = np.linalg.inv(P)
    for i in range(8):
        for j in range(8):
            if abs(P_inv[i,j].real)<=1e-10:
                P_inv[i,j] = (P_inv[i,j] - P_inv[i,j].conjugate())/2
            elif abs(P_inv[i,j].imag)<=1e-10:
                P_inv[i,j] = (P_inv[i,j] + P_inv[i,j].conjugate())/2

    ##initial state
    V[detune,0,:] = np.array([NA*sFo[0,0,0],NA*sFo[0,0,1],NA*sFo[0,0,2],NA*sJa[0,0,0],NA*sJa[0,0,1],NA*sJa[0,0,2],NA*sJa[0,0,3],NA*sJa[0,0,4],NL2/2,0,0,NL2/2])
    Vdel = np.zeros((8,5,5), dtype=np.complex128)
    for i in range(8):
        if i<=2:
            Vdel[i,:,:] = sFo[:,:,i] - V[detune,0,i]*np.eye(5)/NA
        elif i>=3:
            Vdel[i,:,:] = sJa[:,:,i-3] - V[detune,0,i]*np.eye(5)/NA
    for i in range(12):
        for j in range(12):
            if i<=7 and j<=7:
                GammaV[detune,0,i,j] = NA*(np.dot(Vdel[i,:,:], Vdel[j,:,:])[0,0] + np.dot(Vdel[j,:,:], Vdel[i,:,:])[0,0]) / 2
            elif i>=8 and i<=10 and j>=8 and j<=10:
                GammaV[detune,0,i,i] = NL2/4
    L = np.zeros((npulse+1,8),dtype=np.float64)
    L[0,:] = V[detune,0,:8]
    GammaL = np.zeros((npulse+1,8,8), dtype=np.float64)
    GammaL[0,:,:] = GammaV[detune,0,:8,:8]
    #print(V[0,:])
    #print(GammaV[0,:,:])
    VV[detune,:,:] = V[detune,:,:]
    GammaVV[detune,:,:,:] = GammaV[detune,:,:,:]

    time[detune,0] = 0
    X = np.zeros((8,8),dtype=np.complex128)
    for j in range(8):
        X[j,j] = math.cos(-B0*muB*LgS*a[j].imag*period2/hbar)+1j*math.sin(-B0*muB*LgS*a[j].imag*period2/hbar)
    U = np.dot(np.dot(P, X),P_inv)
    U = U.astype(np.float64)
    U_T = U.T
    #print(U)
    #print(U_T)
    Y = np.zeros((8,8),dtype=np.complex128)
    for j in range(8):
        Y[j,j] = math.cos(-B0*muB*LgS*a[j].imag*tau/hbar)+1j*math.sin(-B0*muB*LgS*a[j].imag*tau/hbar)
    U2 = np.dot(np.dot(P, Y),P_inv)
    U2 = U2.astype(np.float64)
    U2_T = U2.T

    ##calculation
    for i in range(0, npulse):
        L[i,:] = V[detune,i,:8]
        GammaL[i,:,:] = GammaV[detune,i,:8,:8]
        L[i,:] = np.dot(U, L[i,:])
        GammaL[i,:,:] = np.dot(np.dot(U, GammaL[i,:,:]), U_T)
        time[detune,i+1] = time[detune,i] + period2 + width
        V[detune,i+1,:8] = L[i,:]
        V[detune,i+1,8:12] = V[detune,0,8:12]
        GammaV[detune,i+1,:8,:8] = GammaL[i,:,:]
        GammaV[detune,i+1,8:12,8:12] = GammaV[detune,0,8:12,8:12]
        V2 = np.zeros((2*nframe+2,12), dtype=np.float64)
        V2[0,:] = V[detune,i+1,:]
        GammaV2 = np.zeros((2*nframe+2,12,12), dtype=np.float64)
        GammaV2[0,:,:] = GammaV[detune,i+1,:,:]
        for j in range(0, 2*nframe, 2):
            V2[j+1,:8] = np.dot(U2, V2[j,:8])
            V2[j+1,8:12] = V[detune,0,8:12]
            GammaV2[j+1,:8,:8] = np.dot(np.dot(U2, GammaV2[j,:8,:8]), U2_T)
            GammaV2[j+1,8:12,8:12] = GammaV[detune,0,8:12,8:12]
            T = np.eye(12)
            for k in range(12):
                V2[j+2,k] = V2[j+1,k] + np.dot(np.dot(V2[j+1,:], Hla[k,:,:]), V2[j+1,:])
                for l in range(12):
                    T[k,l] += np.dot(V2[j+1,:], Hla[k,:,l]) + np.dot(Hla[k,l,:], V2[j+1,:])
            GammaV2[j+2,:,:] = np.dot(np.dot(T, GammaV2[j+1,:,:]), T.T)
            #print(GammaV2[j+2,9,9])
            GammaV3 = np.zeros((12,12), dtype=np.float64)
            for k in range(12):
                for l in range(12):
                    GammaV3[k,l] = GammaV2[j+2,k,9] * GammaV2[j+2,l,9]
            GammaV2[j+2,:,:] = GammaV2[j+2,:,:] - GammaV3[:,:] / GammaV2[j+2,9,9]
            V2[j+2,8:12] = V[detune,0,8:12]
            for k in range(12):
                for l in range(12):
                    if k>=8 or l>=8:
                        GammaV2[j+2,k,l] = 0
            GammaV2[j+2,8:12,8:12] = GammaV[detune,0,8:12,8:12]
        V[detune,i+1,:] = V2[2*nframe,:]
        GammaV[detune,i+1,:,:] = GammaV2[2*nframe,:,:]


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:64: ComplexWarning:

Casting complex values to real discards the imaginary part

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:74: ComplexWarning:

Casting complex values to real discards the imaginary part

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:91: ComplexWarning:

Casting complex values to real discards the imaginary part

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:99: ComplexWarning:

Casting complex values to real discards the imaginary part



In [ ]:
for detune in range(101):
    # detuning
#    Delta_exp = 2*pi * (-1)*(detune+50)*1e7 # in rad/s
#    Delta1 = Delta_exp + 2*pi*509e6
#    Delta2 = Delta_exp - 2*pi*305e6
    period = (20 + (detune*0.5))*1e-6 # period of probe pulses in s
    period2 = period / nframeB

    ##coupling constants G1 G2
    g = weg / (2*eps0*vol)
    alp0 = 3*decay*eps0*hbar*lamJ*lamJ*lamJ/(8*pi*pi)
    alp11 = alp0*abs(wigner_6j(1,J,J,I,F21,F, prec=32))*abs(wigner_6j(1,J,J,I,F21,F, prec=32))
    alp12 = alp0*abs(wigner_6j(1,J,J,I,F22,F, prec=32))*abs(wigner_6j(1,J,J,I,F22,F, prec=32))
    alp21 = -1*alp12*(2*F+1)/(F*(F+1))/Delta2 - alp11*(2*F-1)/F/Delta1
    alp22 = -1*alp12*(2*F+1)/(F*(F+1))/Delta2 + alp11/F/Delta1
    G1 = g*alp21*1e20
    G2 = g*alp22*1e20

    ##light-atom interaction Hamiltonian
    Hla = np.zeros((12,12,12), dtype=np.float64)
    for i in range(12):
        for j in range(12):
            for k in range(12):
                if j == 10:
                    Hla[i,j,k] = G1*stcon[i,2,k]
                elif k == 2:
                    Hla[i,j,k] = G1*stcon[i,10,j]
                elif j == 8:
                    Hla[i,j,k] = G2*stcon[i,3,k]
                elif j == 9:
                    Hla[i,j,k] = G2*stcon[i,4,k]
                elif j == 11:
                    Hla[i,j,k] = G2*stcon[i,7,k]
                elif k == 3:
                    Hla[i,j,k] = G2*stcon[i,8,j]
                elif k == 4:
                    Hla[i,j,k] = G2*stcon[i,9,j]

    a_pre, P = np.linalg.eig(A)
    a = (a_pre - a_pre.conjugate())/2
    for i in range(8):
        for j in range(8):
            if abs(P[i,j].real)<=1e-10:
                P[i,j] = (P[i,j] - P[i,j].conjugate())/2
            elif abs(P[i,j].imag)<=1e-10:
                P[i,j] = (P[i,j] + P[i,j].conjugate())/2

    P_inv = np.linalg.inv(P)
    for i in range(8):
        for j in range(8):
            if abs(P_inv[i,j].real)<=1e-10:
                P_inv[i,j] = (P_inv[i,j] - P_inv[i,j].conjugate())/2
            elif abs(P_inv[i,j].imag)<=1e-10:
                P_inv[i,j] = (P_inv[i,j] + P_inv[i,j].conjugate())/2

    ##initial state
    L = np.zeros((npulse+1,8),dtype=np.float64)
    L[0,:] = VV[detune,0,:8]
    GammaL = np.zeros((npulse+1,8,8), dtype=np.float64)
    GammaL[0,:,:] = GammaVV[detune,0,:8,:8]
    #print(V[0,:])
    #print(GammaV[0,:,:])
    #VV = np.zeros((101,npulse+1,12), dtype=np.float64) 
    #VV[detune,:,:] = V[detune,:,:]
    #GammaVV = np.zeros((101,npulse+1,12,12), dtype=np.float64)
    #GammaVV[detune,:,:,:] = GammaV[detune,:,:,:]

    #time = np.zeros(101,npulse+1)
    #time[detune,0] = 0
    X = np.zeros((8,8),dtype=np.complex128)
    for j in range(8):
        X[j,j] = math.cos(-B0*muB*LgS*a[j].imag*period2/hbar)+1j*math.sin(-B0*muB*LgS*a[j].imag*period2/hbar)
    U = np.dot(np.dot(P, X),P_inv)
    U = U.astype(np.float64)
    U_T = U.T
    #print(U)
    #print(U_T)
    Y = np.zeros((8,8),dtype=np.complex128)
    for j in range(8):
        Y[j,j] = math.cos(-B0*muB*LgS*a[j].imag*tau/hbar)+1j*math.sin(-B0*muB*LgS*a[j].imag*tau/hbar)
    U2 = np.dot(np.dot(P, Y),P_inv)
    U2 = U2.astype(np.float64)
    U2_T = U2.T

    ##calculation 2
    for i in range(0, npulse):
        L[i,:] = VV[detune,i,:8]
        GammaL[i,:,:] = GammaVV[detune,i,:8,:8]
        L[i,:] = np.dot(U, L[i,:])
        GammaL[i,:,:] = np.dot(np.dot(U, GammaL[i,:,:]), U_T)
        VV[detune,i+1,:8] = L[i,:]
        VV[detune,i+1,8:12] = VV[detune,0,8:12]
        GammaVV[detune,i+1,:8,:8] = GammaL[i,:,:]
        GammaVV[detune,i+1,8:12,8:12] = GammaVV[detune,0,8:12,8:12]
        V2 = np.zeros((2*nframe+2,12), dtype=np.float64)
        V2[0,:] = VV[detune,i+1,:]
        GammaV2 = np.zeros((2*nframe+2,12,12), dtype=np.float64)
        GammaV2[0,:,:] = GammaVV[detune,i+1,:,:]
        for j in range(0, 2*nframe, 2):
            V2[j+1,:8] = np.dot(U2, V2[j,:8])
            V2[j+1,8:12] = VV[detune,0,8:12]
            GammaV2[j+1,:8,:8] = np.dot(np.dot(U2, GammaV2[j,:8,:8]), U2_T)
            GammaV2[j+1,8:12,8:12] = GammaVV[detune,0,8:12,8:12]
            T = np.eye(12)
            for k in range(12):
                V2[j+2,k] = V2[j+1,k] + np.dot(np.dot(V2[j+1,:], Hla[k,:,:]), V2[j+1,:])
                for l in range(12):
                    T[k,l] += np.dot(V2[j+1,:], Hla[k,:,l]) + np.dot(Hla[k,l,:], V2[j+1,:])
            GammaV2[j+2,:,:] = np.dot(np.dot(T, GammaV2[j+1,:,:]), T.T)
        VV[detune,i+1,:] = V2[2*nframe,:]
        GammaVV[detune,i+1,:,:] = GammaV2[2*nframe,:,:]


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:74: ComplexWarning:

Casting complex values to real discards the imaginary part

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:82: ComplexWarning:

Casting complex values to real discards the imaginary part



In [ ]:
#Vg = V.astype(np.float64)
#Lgraph = L.astype(np.float64)
#print(V[:,2])
#print(time)

In [ ]:
#def func(x, a, b, c, d, e):
#    return a*np.exp(-b*x)*np.cos(c*x+d) + e

#parameter = np.array([1.0,1.0,1.0,0.0,0.0])
#popt, pcov = curve_fit(func, time, V[:,2])
#print(popt)
#xdata = np.linspace(0, time[npulse], 10*(npulse+1))

In [ ]:
trace0 = go.Scatter(x=time[0,:], y=V[0,:,0], name="Fx", mode='lines+markers')
trace1 = go.Scatter(x=time[0,:], y=V[0,:,1], name="Fy", mode='lines+markers')
trace2 = go.Scatter(x=time[0,:], y=V[0,:,2], name="Fz", mode='lines+markers')
trace3 = go.Scatter(x=time[0,:], y=V[50,:,3], name="Jx", mode='lines+markers')
trace4 = go.Scatter(x=time[0,:], y=V[50,:,4], name="Jy", mode='lines+markers')
trace5 = go.Scatter(x=time[0,:], y=V[50,:,5], name="Jk", mode='lines+markers')
trace6 = go.Scatter(x=time[0,:], y=V[50,:,6], name="Jl", mode='lines+markers')
trace7 = go.Scatter(x=time[0,:], y=V[50,:,7], name="Jm", mode='lines+markers')
#trace1 = go.Scatter(x=xdata, y=func(xdata, *popt), name="sin")#, mode='markers')
var0 = go.Scatter(x=time[0,:], y=GammaVV[50,:,0,0], name="GammaFx", mode='lines+markers')
var1 = go.Scatter(x=time[0,:], y=GammaVV[50,:,1,1], name="GammaFy", mode='lines+markers')
var2 = go.Scatter(x=time[0,:], y=GammaVV[50,:,2,2], name="GammaFz", mode='lines+markers')
var3 = go.Scatter(x=time[0,:], y=GammaVV[50,:,3,3], name="GammaJx", mode='lines+markers')
var4 = go.Scatter(x=time[0,:], y=GammaVV[50,:,4,4], name="GammaJy", mode='lines+markers')
var5 = go.Scatter(x=time[0,:], y=GammaVV[50,:,5,5], name="GammaJk", mode='lines+markers')
var6 = go.Scatter(x=time[0,:], y=GammaVV[50,:,6,6], name="GammaJl", mode='lines+markers')
var7 = go.Scatter(x=time[0,:], y=GammaVV[50,:,7,7], name="GammaJm", mode='lines+markers')

sqvar0 = go.Scatter(x=time[0,:], y=GammaV[50,:,0,0], name="GammaFx with squeezing", mode='lines+markers')
sqvar1 = go.Scatter(x=time[0,:], y=GammaV[50,:,1,1], name="GammaFy with squeezing", mode='lines+markers')
sqvar2 = go.Scatter(x=time[0,:], y=GammaV[50,:,2,2], name="GammaFz with squeezing", mode='lines+markers')
sqvar3 = go.Scatter(x=time[0,:], y=GammaV[50,:,3,3], name="GammaJx with squeezing", mode='lines+markers')
sqvar4 = go.Scatter(x=time[0,:], y=GammaV[50,:,4,4], name="GammaJy with squeezing", mode='lines+markers')
sqvar5 = go.Scatter(x=time[0,:], y=GammaV[50,:,5,5], name="GammaJk with squeezing", mode='lines+markers')
sqvar6 = go.Scatter(x=time[0,:], y=GammaV[50,:,6,6], name="GammaJl with squeezing", mode='lines+markers')
sqvar7 = go.Scatter(x=time[0,:], y=GammaV[50,:,7,7], name="GammaJm with squeezing", mode='lines+markers')


layout = go.Layout(
    title="time evolution for spin orientation",
    #legend={"x":0.8, "y":0.1},
    xaxis={"title":"time(s)"},
    yaxis={"title":"spin value"},
    updatemenus=[dict(
            type="buttons",
            buttons=[dict(label="Play",
                          method="animate",
                          args=[None])])]
)

#frames=[go.Frame(
#        data=[go.Scatter(x=time[k,:],y=V[k,:,2], name="Fz")for k in range(1, 101)])]

fig = go.Figure(data = [trace0,trace1,trace2], layout = layout)
plotly.offline.iplot(fig)

In [ ]:
# Create figure
fig = go.Figure()

# Add traces, one for each slider step
for step in np.arange(101):
    fig.add_trace(
        go.Scatter(
            visible=False,
            mode='lines+markers',
#            line=dict(color="#00CED1", width=6),
            name="period = " + str(((step*0.5)+20) * 1e-6),
            x=time[step,:],
            y=GammaV[step,:,2,2],
            showlegend = True))

# Make 10th trace visible
fig.data[10].visible = True

# Create and add slider
steps = []
for i in range(len(fig.data)):
    step = dict(
        method="update",
        args=[{"visible": [False] * len(fig.data)},
              {"title": "Fz in period  step: " + str(i)}],  # layout attribute
    )
    step["args"][0]["visible"][i] = True  # Toggle i'th trace to "visible"
    steps.append(step)

sliders = [dict(
    active=10,
    currentvalue={"prefix": "period: "},
    pad={"t": 50},
    steps=steps
)]

fig.update_layout(
    sliders=sliders
)
fig.show()
#plotly.offline.plot(fig)#, filename='Fz in period.html')

In [ ]:
layout = go.Layout(
    title="time evolution for spin alignment",
    #legend={"x":0.8, "y":0.1},
    xaxis={"title":"time(s)"},
    yaxis={"title":"spin value"},
)
fig = go.Figure(data = [trace3,trace4,trace5,trace6,trace7], layout = layout)
plotly.offline.iplot(fig)

In [ ]:
layout = go.Layout(
    title="time evolution for spin orientation variance",
    #legend={"x":0.8, "y":0.1},
    xaxis={"title":"time"},
    yaxis={"title":"variance"},
)
fig = go.Figure(data = [var0,var1,var2], layout = layout)
plotly.offline.iplot(fig)

In [ ]:
layout = go.Layout(
    title="time evolution for spin alignment variance",
    #legend={"x":0.8, "y":0.1},
    xaxis={"title":"time"},
    yaxis={"title":"variance"},
)
fig = go.Figure(data = [var3,var4,var5,var6,var7], layout = layout)
plotly.offline.iplot(fig)

In [ ]:
layout = go.Layout(
    title="time evolution for spin component and variance",
    #legend={"x":0.8, "y":0.1},
    xaxis={"title":"time"},
    yaxis={"title":"Fz"},
)
fig = go.Figure(data = [trace2,trace3], layout = layout)
plotly.offline.iplot(fig)

In [ ]:
max_GammaFz = np.zeros(101, dtype=np.float64)
max_GammaFz2 = np.zeros(101, dtype=np.float64)
diff_GammaFz = np.zeros(101, dtype=np.float64)
for i in range(101):
    max_GammaFz[i] = np.max(GammaV[i,:,2,2])
    max_GammaFz2[i] = np.max(GammaVV[i,:,2,2])
    diff_GammaFz[i] = max_GammaFz2[i] - max_GammaFz[i]

In [ ]:
freq = -1 * (np.arange(101) + 50) *10
print(freq)

[ -500  -510  -520  -530  -540  -550  -560  -570  -580  -590  -600  -610
  -620  -630  -640  -650  -660  -670  -680  -690  -700  -710  -720  -730
  -740  -750  -760  -770  -780  -790  -800  -810  -820  -830  -840  -850
  -860  -870  -880  -890  -900  -910  -920  -930  -940  -950  -960  -970
  -980  -990 -1000 -1010 -1020 -1030 -1040 -1050 -1060 -1070 -1080 -1090
 -1100 -1110 -1120 -1130 -1140 -1150 -1160 -1170 -1180 -1190 -1200 -1210
 -1220 -1230 -1240 -1250 -1260 -1270 -1280 -1290 -1300 -1310 -1320 -1330
 -1340 -1350 -1360 -1370 -1380 -1390 -1400 -1410 -1420 -1430 -1440 -1450
 -1460 -1470 -1480 -1490 -1500]


In [ ]:
score0 = go.Scatter(x=pulse_period[:], y=max_GammaFz[:], name="max_GammaFz with squeezing", showlegend = True, mode='lines+markers')
score1 = go.Scatter(x=pulse_period[:], y=max_GammaFz2[:], name="max_GammaFz", showlegend = True, mode='lines+markers')
score2 = go.Scatter(x=pulse_period[:], y=diff_GammaFz[:], name="difference between squeezing and nonsqueezing", showlegend = True, mode='lines+markers')
layout = go.Layout(
        #title=dict(text =f'Silhouette and Davies Bouldin/{name}', font =dict(family='Sherif', size=30),x=0.5, y=1),
        title={
                'text': "max variance in pulse period domain",
                'y':0.98,
                'x':0.5,
        #         'xanchor': 'center',
                'yanchor': 'top',
                "font": {
                    "family":"Arial",
                    "color":"black",
                    "size": 20
                  },
                 },
        xaxis = dict(title = "period(s)"),
        yaxis = dict(title = "variance", side = 'left', showgrid=True),
        #yaxis2 = dict(title = 'pseudo_F', side = 'right',overlaying = 'y', range = [0, max(c_index)], showgrid=False),
        #yaxis3 = dict(title = 'Davies_Bouldin', side = 'right',overlaying = 'y', range = [0, max(d_index)], showgrid=False),
        legend=dict(
                x=1,
                y=0,
                traceorder="normal",
                font=dict(
                    family="sans-serif",
                    size=10,
                    color="black"
                    ),
                bgcolor='rgba(0,0,0,0)',#"LightSteelBlue",
        #         bordercolor="Black",
        #         borderwidth=2
                ),
        width=800,
        height=500,
        font=dict(
        #         family="Courier New, monospace",
                    size=10,
        #         color="#7f7f7f"
                    ),
        margin=dict(l=0, r=0, b=0, t=40),
    )

fig = go.Figure(data = [score0, score1, score2], layout = layout)
plotly.offline.iplot(fig)